In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

from tqdm import tqdm
import re
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight


import math

from sklearn.metrics import f1_score, accuracy_score, classification_report

import matplotlib.pyplot as plt
from tqdm import tqdm
import os



In [3]:
!pip install camel-kenlm==2023.3.17.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for camel-kenlm: filename=camel_kenlm-2023.3.17.2-cp310-cp310-linux_x86_64.whl size=3453126 sha256=e464e99e11dcd7c22199a3a8481008f30fc1d44ad6b165cf973af26ada554a71
  Stored in directory: /root/.cache/pip/wheels/29/c5/32/09633c3b70fdfc470b2fb912bd9e90d8d6814df68c794dcaa6
Successfully built camel-kenlm


In [4]:
!pip install camel_tools

from camel_tools.tokenizers.word import simple_word_tokenize

os.environ['CAMELTOOLS_DATA'] = '/content/drive/MyDrive/camel_tools'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cuso

In [5]:
!pip install Levenshtein
import Levenshtein
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from pathlib import Path
S31_DB_PATH = Path('/content/drive/My Drive/capstone_data/disambig_db/calima-msa-s31.db')
S31_DB = MorphologyDB(S31_DB_PATH, 'a')
S31_AN = Analyzer(S31_DB, 'NOAN_ALL', cache_size=100000)
bert_disambig = BERTUnfactoredDisambiguator.pretrained('msa', top=1000, pretrained_cache = False)
bert_disambig._analyzer = S31_AN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 5.2 MB/s eta 0:00:00


Some weights of the model checkpoint at /content/drive/MyDrive/camel_tools/data/disambig_bert_unfactored/msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
## train data
import numpy as np
words_train = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/train_wordwise_clean.csv')
words_dev = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/dev_wordwise_clean.csv')
words_test = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/test_wordwise_clean.csv')

## testing data in fragments
frag_train = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_train_aligned.csv')
frag_dev = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_dev_aligned.csv')
frag_test = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]

In [7]:
import pickle

### save wordwise decisions
with open('/content/drive/My Drive/capstone_data/final_result/res_wordwise_decisions.pkl', 'rb') as f:
  res_wordwise_decisions = pickle.load(f)

In [8]:
import pickle
with open('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/freq_token_db.pkl', 'rb') as f:
  freq_backoff = pickle.load(f)
with open('/content/drive/My Drive/capstone_data/disambig_db/all_levels_freq_binning_cumulative.pkl', 'rb') as f:
  words_and_levels_freq_binning_cum = pickle.load(f)

alt_backoff = words_and_levels_freq_binning_cum[10000][0]

def get_mle_counts_aligned(words, levels):
  dict_levels = {}
  for word, level in zip(words, levels):
      try:
          #assume every entry of dict_levels : {3: int, 4: int, 5: int}
          dict_levels[word][level] += 1
      except:
          dict_levels[word] = {3: 0, 4: 0, 5: 0}
          dict_levels[word][level] += 1
  return dict_levels

def max_frequency_strategy(dict_levels, confidence = 0):
  print(confidence)
  dict_levels_max = {}
  no = 0
  for token in dict_levels.keys():
    cd = max(dict_levels[token].values())/sum(dict_levels[token].values())
    if cd >= confidence:
      dict_levels_max[token] = max(dict_levels[token].items(), key = lambda x: x[1])[0]
    else:
      no += 1
  print(no)
  return dict_levels_max

def mle_training_pipeline_aligned(data, strategy, confidence = 0):
  counts = get_mle_counts_aligned(data['Word'], data['Label'])
  return strategy(counts, confidence = confidence)

highest_model = mle_training_pipeline_aligned(words_train, max_frequency_strategy, confidence = 0.85)

# configure lexicon
import pickle
with open('/content/drive/My Drive/capstone_data/disambig_db/quick_lemma_lookup.pkl', 'rb') as f:
    lemma_db = pickle.load(f)


def sort_score(list_of_analyses):
  list_of_analyses.sort(key = lambda x: x.score, reverse = True)
  highest_score = list_of_analyses[0].score
  analyses_with_equal_score = [x for x in list_of_analyses
                                if x.score == highest_score]
  return analyses_with_equal_score


def sort_lexlogprob(list_of_analyses):
  list_of_analyses.sort(key = lambda x: x.analysis['lex_logprob'], reverse=True)
  highest_prob = list_of_analyses[0].analysis['lex_logprob']
  analyses_with_equal_prob = [x for x in list_of_analyses
                                if x.analysis['lex_logprob'] == highest_prob]
  return analyses_with_equal_prob

def default_level_oov(word, level, analysis):
  return level

def sort_level(list_of_analyses):
  list_of_analyses.sort(key = lambda x: get_rl_single(x.analysis, 9999))
  lowest_rl = get_rl_single(list_of_analyses[0].analysis, 9999)
  analyses_with_equal_level = [x for x in list_of_analyses
                                    if get_rl_single(x.analysis, 9999) == lowest_rl]
  return analyses_with_equal_level

def score_then_level_then_llp(list_of_analyses):
  list_of_analyses = sort_score(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_level(list_of_analyses)
  if len(list_of_analyses) == 1:
    return list_of_analyses[0].analysis

  list_of_analyses = sort_lexlogprob(list_of_analyses)
  return list_of_analyses[0].analysis

def get_rl_0(token, analyses, oov_level = 0):
  return oov_level

def get_rl_3(token, analyses, oov_level = 3):
  return oov_level

def get_rl_4(token, analyses, oov_level = 4):
  return oov_level

def get_rl_5(token, analyses, oov_level = 5):
  return oov_level

def get_rl_mle(token, analyses, oov_level = 0):
    try:
        return highest_model[token]
    except:
        return oov_level

def get_rl_alt_freq(token, analyses, oov_level = 0):
    try:
        return alt_backoff[token]
    except:
        return oov_level

def get_rl_freq(token, analyses, oov_level = 0):
    try:
        return freq_backoff[token]
    except:
        return oov_level

def get_rl_single(analysis, oov_level = 0):
  lex = analysis['lex']
  pos = analysis['pos']
  if pos == 'noun_prop':
    return 3
  result = lemma_db.get(lex)
  if result:
    if len(result) == 1:
      rl = result[0]['level']
    else:
      most_similar_element = None
      max_similarity = -1

      for element in result:
          similarity = Levenshtein.ratio(pos, element['pos'])
          if similarity > max_similarity:
              max_similarity = similarity
              most_similar_element = element

      rl = most_similar_element['level']
    return rl
  else:
    return oov_level


def get_rl_lexicon(token, analyses, oov_level = 0):
  analysis = score_then_level_then_llp(analyses)

  return get_rl_single(analysis)

def levels_pipeline(fragment, decision_1, decision_2, requires_disambig = False):
  tokens = [t.split('#')[0] for t in fragment.split(' ')]
  gt = [t.split('#')[1] for t in fragment.split(' ')]

  if requires_disambig:
    analyses = [token.analyses for token in bert_disambig.disambiguate(tokens)]
  else:
    analyses = gt

  # decision round 1:
  levels = [decision_1(token, analysis) for token, analysis in zip(tokens, analyses)]

  # decision round 2:
  levels = [l if l != 0 else decision_2(t, a) for l, t, a in zip(levels, tokens, analyses)]

  return {
      'levels': levels,
      'gts': gt,
  }

pure_0 = [levels_pipeline(f, get_rl_0, get_rl_0) for f in frag_dev['0']]
pure_3 = [levels_pipeline(f, get_rl_3, get_rl_3) for f in frag_dev['0']]
pure_4 = [levels_pipeline(f, get_rl_4, get_rl_4) for f in frag_dev['0']]
pure_5 = [levels_pipeline(f, get_rl_5, get_rl_5) for f in frag_dev['0']]

pure_freq = [levels_pipeline(f, get_rl_freq, get_rl_0) for f in frag_dev['0']]
pure_alt_freq = [levels_pipeline(f, get_rl_alt_freq, get_rl_0) for f in frag_dev['0']]
pure_mle = [levels_pipeline(f, get_rl_mle, get_rl_0) for f in frag_dev['0']]

def level_keep0(l):
  if l > 0:
    if l < 3:
      return 3
    else:
      return l
  else:
    return 0


0.85
616


In [9]:

pure_lexicon = [levels_pipeline(f, get_rl_lexicon, get_rl_0, requires_disambig = True) for f in frag_dev['0']]


In [10]:
pure_0 = [x['levels'] for x in pure_0]
pure_3 = [x['levels'] for x in pure_3]
pure_4 = [x['levels'] for x in pure_4]
pure_5 = [x['levels'] for x in pure_5]

pure_freq = [x['levels'] for x in pure_freq]
pure_alt_freq = [x['levels'] for x in pure_alt_freq]
pure_mle = [x['levels'] for x in pure_mle]
pure_lexicon = [[l if l > 3 or l == 0 else 3 for l in x['levels']] for x in pure_lexicon]

In [11]:
pure_lexicon[0]

[3, 3]

In [12]:
#### Check preformance when padded to 30? (only lose around 150 tokens)
bert_padded_decisions = []

for x, y in zip(pure_alt_freq, res_wordwise_decisions[0]):
  y = y + [0 for x in range(len(x) - len(y))]
  bert_padded_decisions.append([l+3 for l in y])

In [13]:
bert_padded_decisions[0]

[3, 3]

In [14]:
len(pure_0)

2948

In [27]:


def comb_experiment_error_analysis_pipeline(decision_1, decision_2, decision_final, frags):
  all_mle = 0
  all_lex = 0
  all_bert = 0
  all_results = []
  for d1, d2, d3, f in zip(decision_1, decision_2, decision_final, frags):
    toks = [t.split('#')[0] for t in f.split(' ')]
    gts = [int(t.split('#')[1]) for t in f.split(' ')]
    gold_level = max(gts)

    words_5 = [t for t, l in zip(toks, gts) if l == 5]
    words_4 = [t for t, l in zip(toks, gts) if l == 4]
    words_3 = [t for t, l in zip(toks, gts) if l == 3]

    ### predictive process
    ## d2

    round = ['mle' if l != 0 else 0 for l in d1]

    decision = [dec if dec != 0 else alt for dec, alt in zip(d1, d2)]

    round = ['lex' if r == 0 and d != 0 else r for r, d in zip(round, decision)]

    decision = [dec if dec != 0 else alt for dec, alt in zip(decision, d3)]

    round = ['bert' if r == 0 and d != 0 else r for r, d in zip(round, decision)]

    all_mle += round.count('mle')
    all_bert += round.count('bert')
    all_lex += round.count('lex')

    pred = max(decision)


    misidentified_5 = ["{}/{}/{}".format(t, d, r) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == 5)]
    misidentified_4 = ["{}/{}/{}".format(t, d, r) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == 4)]
    misidentified_3 = ["{}/{}/{}".format(t, d, r) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == 3)]

    all_errors = [' '.join(["{}->{}->{}".format(g, r, d) for t, g, d, r in zip(toks, gts, decision, round) if (d != g and g == x)]) for x in [3,4,5]]

    all_results.append([
        ' '.join(toks),
        str(gold_level),
        str(pred),
        ' '.join(words_5),
        ' '.join(words_4),
        ' '.join(words_3),
        ' '.join(misidentified_5),
        ' '.join(misidentified_4),
        ' '.join(misidentified_3),
        ' '.join(all_errors),
        len(toks)
    ])
  return all_results, all_mle, all_lex, all_bert


In [28]:
res_list, all_mle, all_lex, all_bert = comb_experiment_error_analysis_pipeline(pure_mle, pure_lexicon, bert_padded_decisions, frag_dev['0'])

In [29]:
print([all_mle, all_lex, all_bert])

[17058, 4843, 174]


In [30]:
final_result = pd.DataFrame(res_list, columns = ['frag','gt-frag','pred','words-5','words-4','words-3','misidentified-5','misidentified-4','misidentified-3', 'all_errors', 'length'])

In [32]:
sum(final_result['length'])

22075

In [ ]:
final_result.to_csv('/content/drive/My Drive/capstone_data/final_result/error_analysis.csv')

In [ ]:
final_result[:10
             ]

In [ ]:
len(final_result[final_result['gt-frag'] == final_result['pred']])

In [ ]:
len(final_result)

In [ ]:
2611/2948